In [1]:
import yahooquery as yq
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools
import operator
import kaleido
from datetime import timedelta, datetime

import warnings
warnings.filterwarnings("ignore")

In [3]:
# put images into 'images' folder in the same directory
def ants_indicator(tickers:list, period:str='1y', purpose:str='present', verbose:bool=True, **kwargs) -> list:
    buy_list = []  # final result

    main_df = yq.Ticker(tickers, formatted=True, validate=True).history(period=period, interval="1d")
    main_df = main_df.reset_index()
    main_df['date'] = [datetime.strptime(x.strftime('%Y-%m-%d'), ('%Y-%m-%d')) for x in main_df['date']]

    for n, ticker in enumerate(tickers):
        df = main_df[main_df['symbol'] == ticker]
        df.reset_index(drop=True, inplace=True)

        df['momentum'] = df['adjclose'] > df['adjclose'].shift(1)
        df['momentum'] = [df['momentum'][x:15+x].value_counts()[True] if True in df['momentum'][x:15+x].value_counts().keys() 
                        else 0 for x in range(len(df))]

        df['avg_volume'] = df['volume'].rolling(30).mean()

        df['last_15_price'] = df['adjclose'].shift(15)

        # momentum only
        grey_index = set()
        for x in range(len(df) - 15):
            if df['momentum'][x] >= 12:
                grey_index.add(x)

        # momentum and volume
        orange_index = set()
        for x in grey_index:
            if df['volume'][x:15+x].mean() > df['avg_volume'][x] * 1.25:
                orange_index.update(range(x, x+15))

        # momentum and price
        blue_index = set()
        for x in grey_index:
            if df['adjclose'][x:15+x].mean() > df['last_15_price'][x] * 1.2:
                blue_index.update(range(x, x+15))

        green_index = orange_index.intersection(blue_index)

        # add on for momentum only
        temp = grey_index.copy()
        for x in temp:
            grey_index.update(range(x, x+15))

        # make sure indexes are not duplicated
        blue_index -= green_index
        orange_index -= green_index
        grey_index = grey_index - green_index - blue_index - orange_index

        indexes = [grey_index, blue_index, orange_index, green_index]
        new_indexes = []

        for index in indexes:
            consec_count = [0] + list(itertools.accumulate([sum(1 for _ in g) for _, g in 
                                                            itertools.groupby([e-i for i, e in enumerate(sorted(list(index)))])], operator.add))
            temp = sorted(list(index.copy()))
            cnt = 0
            for i in range(1, len(consec_count)):
                if consec_count[i] - consec_count[i-1] < 15:
                    del temp[consec_count[i-1] - cnt : consec_count[i] - cnt]
                    cnt += consec_count[i] - consec_count[i-1]
            new_indexes.append(temp)

        buy_signal = ""
        if purpose == 'present':
            if len(new_indexes[3]) > 0:
                if max(new_indexes[3]) > len(df) - 30:
                    buy_signal = "[BUY]"
                    buy_list.append([ticker,
                                    df['date'][max(new_indexes[3]).strftime('%Y-%m-%d')],
                                    round(df['adjclose'][max(new_indexes[3])], 4)])
              
        elif purpose == 'backtest':  # require at least 2 years of data
            try:
                if len(new_indexes[3]) > 0 and len(df) >= 500:
                    if min(new_indexes[3])+15 < 500:  # get the oldest green ant trend
                        buy_signal = "[BUY]"
                        # ticker, green ant date, 1yo price, current price, returns
                        buy_list.append([ticker, 
                                        df['date'][min(new_indexes[3])+15].strftime('%Y-%m-%d'),
                                        round(df['adjclose'][min(new_indexes[3])+15], 4), 
                                        round(df['adjclose'][min(new_indexes[3])+15+252], 4)],
                                        (round(df['adjclose'][min(new_indexes[3])+15+252], 4) - round(df['adjclose'][min(new_indexes[3])+15], 4)) 
                                        / round(df['adjclose'][min(new_indexes[3])+15], 4))
            except:
                pass
    
        ant_dates, ant_price = [[] for _ in range(4)], [[] for _ in range(4)]

        for i in range(4):
            for x in new_indexes[i]:
                ant_dates[i].append(df['date'][x])
                ant_price[i].append(df['high'][x] * 1.03)

        missing_dates = sorted(set(df.date[0] + timedelta(x) for x in range((df.date[len(df.date)-1] - df.date[0]).days)) - set(df.date))

        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.03, row_width=[0.2, 0.7])
        
        # price candlestick
        fig.add_trace(go.Candlestick(x=df['date'],
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close']),
                        row=1, col=1)
        
        # volume
        fig.add_trace(go.Bar(x=df['date'], y=df['volume'], 
                            showlegend=False,
                            marker_color='skyblue',
                            marker_line=dict(width=1,
                                        color='darkslategrey')), 
                            row=2, col=1)

        fig.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                            label="1m",
                            step="month",
                            stepmode="backward"),
                        dict(count=6,
                            label="6m",
                            step="month",
                            stepmode="backward"),
                        dict(count=1,
                            label="YTD",
                            step="year",
                            stepmode="todate"),
                        dict(count=1,
                            label="1y",
                            step="year",
                            stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True
                ),
                type="date"
            ),
            autosize=False,
            width=1500,
            height=750,
            margin=dict(l=20, r=20, t=20, b=20),
            xaxis_rangeslider_visible=False)

        fig.update_xaxes(
                rangebreaks=[
                    dict(bounds=["sat", "mon"]),
                    dict(values=missing_dates)
                    # dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
                ]
            )

        color = ['grey', 'navy', 'goldenrod', 'lightgreen']

        for i in range(4):
            fig.add_trace(
                go.Scatter(
                    x=ant_dates[i],
                    y=ant_price[i],
                    mode="markers",
                    marker=dict(symbol='circle', 
                                size=6, 
                                color=color[i],
                                line=dict(width=1,
                                            color='darkslategrey'))))

        # fig.show()
        if verbose:
            print(f'{n+1}: {ticker}')
            fig.write_image(f"images/{buy_signal} {ticker}.png", format='png', engine='kaleido')
  
    return buy_list

In [4]:
market_ret = yq.Ticker('SPY', formatted=True, validate=True).history(period='5y', interval='1d')['adjclose']
market_ret = (market_ret[-1] - market_ret[0]) / market_ret[0]
market_ret

0.9793898033677015

In [5]:
tickers = pd.read_csv('large_cap.csv').Symbol.to_list()
result = ants_indicator(tickers=tickers, period='5y', purpose='backtest', verbose=True)

rets = [x[4] for x in result] # returns
stdev = np.std(rets)
sharpe = (rets - market_ret) / stdev

np.savetxt("result.csv",
            result,
            delimiter =", ",
            fmt ='% s')

stdev, sharpe

1: A
2: AAL
3: AAPL
4: ABBV
5: ABEV
6: ABNB
7: ABT
8: ACGL
9: ACI
10: ACM
11: ACN
12: ADBE
13: ADI
14: ADM
15: ADP
16: ADSK
17: AEE
18: AEG
19: AEM
20: AEP
21: AER
22: AES
23: AFG
24: AFL
25: AFRM
26: AGNCL
27: AGNCM
28: AGNCN
29: AGNCO
30: AGNCP
31: AGR
32: AIG
33: AJG
34: AKAM
35: ALB
36: ALC
37: ALGN
38: ALL
39: ALLE
40: ALLY
41: ALNY
42: AMAT
43: AMCR
44: AMD
45: AME
46: AMGN
47: AMH
48: AMP
49: AMT
50: AMX
51: AMZN
52: ANET
53: ANSS
54: AON
55: AOS
56: APD
57: APH
58: APO
59: APOS
60: APP
61: APTV
62: AQNB
63: AQNU
64: ARCC
65: ARE
66: ARES
67: ARGX
68: ARM
69: ASBA
70: ASML
71: ASX
72: ATO
73: AVB
74: AVGO
75: AVTR
76: AVY
77: AWK
78: AXON
79: AXP
80: AZN
81: AZO
82: AZPN
83: BA
84: BABA
85: BAC
86: BAH
87: BALL
88: BAM
89: BAP
90: BAX
91: BBD
92: BBDO
93: BBVA
94: BBWI
95: BBY
96: BCE
97: BCH
98: BCS
99: BDX
100: BEKE
101: BEN
102: BEP
103: BG
104: BGNE
105: BHP
106: BIDU
107: BIIB
108: BIP
109: BK
110: BKDT
111: BKNG
112: BKR
113: BLD
114: BLDR
115: BLK
116: BMO
117: BMRN
118: 

(nan, array([], dtype=float64))